### 필요한것들 다운 (모듈, 크롬옵션)

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin
!pip install webdriver_manager
!pip install chromedriver_autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,326 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,384 kB]
Hit:11 http://archive.ubunt

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat
from itertools import groupby

import pandas as pd
import numpy as np

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # GUI 없는 환경에서는 headless 모드를 사용
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

### [ 필독 ] 코드 설명 및 주의할 점



0. 수정해야 할 부분
- pickle로 저장하는 부분 각자 경로에 맞춰서 수정

1. 코드 설명
  - i, j, k가 각각 의미하는 바는 함수 옆에 적어뒀습니당
  - 이 중에 j가 몇년도인지를 의미하는데, 틈틈이 df.head()로 맞는 연도가 잘 불러와졌는지 확인해주시기를 바랍니다 !
    - 참고) j=1 : 가장 최근 연도 - 즉 2020년대의 경우 2023년 / 2010년대~의 경우 2019년을 의미


2. 주의할 점
  - get_songs 끝에 보면 pickle로 저장할 수 있게 만들어뒀는데, 이 때 저장할 경로를 각자 맞게 수정한 뒤 실행하시기를 바랍니다
  - 또, 모든 연도를 한 번에 돌리면 중간에 멈추는 불상사가 일어날 수도 있기 때문에,, 일부는 실행하고 일부는 주석처리하는 방식으로 진행하는 것을 추천합니다..

3. 첨언하자면 ..
  - 아무래도 노가다로 만들어둔 코드라 숫자 실수가 있을 수도 있습니다.. 연도별로 정확한 i, j, k 값을 명시해뒀으니 참고해서 한 번씩 확인해주시면 감사할 것 같습니다 :) ...

### 최종 코드(함수)

In [13]:
def get_songs_kor(i,j,k): # i:연대, j:연도, k:장르/스타일

    driver = webdriver.Chrome(options = chrome_options)
    driver.get('https://www.melon.com/chart/index.htm')
    time.sleep(3)

  ## -------[특정 연도 불러오기]----------------------------------------------------

    # 차트파인더 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span'))).click()

    # 연도차트 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="d_chart_search"]/div/h4[3]/a'))).click()

    # 연대선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[{i}]/span/label'))).click()

    # 연도선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[{j}]/span/label'))).click()

    # 장르/스타일 선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[{k}]/span/label'))).click()

    # 검색 버튼
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="d_srch_form"]/div[2]/button/span/span'))).click()

    # 페이지 로드 기다리기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((
            By.XPATH, '//*[@id="lst50"]/td[4]/div/div/div[1]/span/strong/a')))

    year = driver.find_element(By.XPATH, '//*[@id="serch_cnt"]/span[1]').text

  ## -------[노래 특성 가져오기]----------------------------------------------------

    # 리스트 리셋
    song_numbers = []
    titles = []
    artists = []
    lyrics = []
    whens = []
    likes = []
    genres = []

    # 각 노래의 고유한 번호 찾고 모으기
    ls50 = driver.find_elements(By.XPATH, '//*[@id="lst50"]/td[5]/div/button')
    ls100 = driver.find_elements(By.XPATH, '//*[@id="lst100"]/td[5]/div/button')

    for i in ls50:
      song_numbers.append(i.get_attribute('data-song-no'))
    for i in ls100:
      song_numbers.append(i.get_attribute('data-song-no'))

    # 노래 번호로 그 노래 상세 페이지 들어가기
    for number in song_numbers:
      driver = webdriver.Chrome(options = chrome_options) ####################
      driver.get(f'https://www.melon.com/song/detail.htm?songId={number}') # 새로운 페이지 오픈
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

      title = driver.find_element(By.CLASS_NAME, 'song_name').text
      artist = driver.find_element(By.CLASS_NAME, 'artist').text

      # 가사 없는 경우 np.nan으로 처리 (find_elements로 진행)
      lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
      lyric =  lyric_element[0].text if lyric_element else np.nan

      when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
      like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text
      genre = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text

      titles.append(title)
      artists.append(artist)
      lyrics.append(lyric)
      whens.append(when[:4])
      likes.append(like)
      genres.append(genre)

    year_rep = np.repeat(year,len(titles))

    df = pd.DataFrame({'연도':year_rep, '제목':titles,'가수':artists, '가사':lyrics,
                      '발매년도':whens, '장르':genres, '좋아요 수':likes})

    df.to_pickle(f'/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 5 - 데이터 수집/DATA/Chart/pickles/한국_{year}.pkl')

    return df

In [ ]:
def get_songs_pop(i,j,k): # i:연대, j:연도, k:장르/스타일

    driver = webdriver.Chrome(options = chrome_options)
    driver.get('https://www.melon.com/chart/index.htm')
    time.sleep(3)

  ## -------[특정 연도 불러오기]----------------------------------------------------

    # 차트파인더 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span'))).click()

    # 연도차트 클릭
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="d_chart_search"]/div/h4[3]/a'))).click()

    # 연대선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[{i}]/span/label'))).click()

    # 연도선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[{j}]/span/label'))).click()

    # 장르/스타일 선택
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, f'//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[{k}]/span/label'))).click()

    # 검색 버튼
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="d_srch_form"]/div[2]/button/span/span'))).click()

    # 페이지 로드 기다리기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((
            By.XPATH, '//*[@id="lst50"]/td[4]/div/div/div[1]/span/strong/a')))

    year = driver.find_element(By.XPATH, '//*[@id="serch_cnt"]/span[1]').text

  ## -------[노래 특성 가져오기]----------------------------------------------------

    # 리스트 리셋
    song_numbers = []
    titles = []
    artists = []
    lyrics = []
    whens = []
    likes = []
    genres = []

    # 각 노래의 고유한 번호 찾고 모으기
    ls50 = driver.find_elements(By.XPATH, '//*[@id="lst50"]/td[5]/div/button')
    ls100 = driver.find_elements(By.XPATH, '//*[@id="lst100"]/td[5]/div/button')

    for i in ls50:
      song_numbers.append(i.get_attribute('data-song-no'))
    for i in ls100:
      song_numbers.append(i.get_attribute('data-song-no'))

    # 노래 번호로 그 노래 상세 페이지 들어가기
    for number in song_numbers:
      driver = webdriver.Chrome(options = chrome_options) ####################
      driver.get(f'https://www.melon.com/song/detail.htm?songId={number}') # 새로운 페이지 오픈
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'song_name')))

      title = driver.find_element(By.CLASS_NAME, 'song_name').text
      artist = driver.find_element(By.CLASS_NAME, 'artist').text

      # 가사 없는 경우 np.nan으로 처리 (find_elements로 진행)
      lyric_element = driver.find_elements(By.CLASS_NAME, 'lyric')
      lyric =  lyric_element[0].text if lyric_element else np.nan

      when = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
      like = driver.find_element(By.XPATH, '//*[@id="d_like_count"]').text
      genre = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]').text

      titles.append(title)
      artists.append(artist)
      lyrics.append(lyric)
      whens.append(when[:4])
      likes.append(like)
      genres.append(genre)

    year_rep = np.repeat(year,len(titles))

    df = pd.DataFrame({'연도':year_rep, '제목':titles,'가수':artists, '가사':lyrics,
                      '발매년도':whens, '장르':genres, '좋아요 수':likes})

    df.to_pickle(f'/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Fall/추천시스템/Week 5 - 데이터 수집/DATA/Chart/pickles/해외_{year}.pkl')

    return df

### 한국 노래들

- [2020년대] -> 400곡
  - i = 1
  - j = 1 ~ 4
  - k = 2

최종 - 400

In [ ]:
# 2020년대 -------------------------------

i = 1
k = 2

df = get_songs_kor(i, 1, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 2, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 3, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 4, k)
print("완료했습니당 :)")

완료했습니당 :)


KeyboardInterrupt: 

- [2010년대] -> 1000곡
  - i = 2
  - j = 1 ~ 10
  - k = 2

최종 - 400 + 1000 = 1400

In [ ]:
# 2010년대 -------------------------------

i = 2
k = 2

df = get_songs_kor(i, 1, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 2, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 3, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 4, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 5, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 6, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 7, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 8, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 9, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 10, k)
print("완료했습니당 :)")

완료!
완료!


- [2000년대] -> 1000곡
  - i = 3
  - j = 1 ~ 5 // 6 ~ 10
  - k = 2 // 1

최종 - 1400 + 1000 = 2400


In [ ]:
# 2000년대 (1) -------------------------------

i = 3
k = 2

df = get_songs_kor(i, 1, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 2, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 3, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 4, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 5, k)
print("완료했습니당 :)")

In [ ]:
# 2000년대 (2) -------------------------------

i = 3
k = 1

df = get_songs_kor(i, 6, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 7, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 8, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 9, k)
print("완료했습니당 :)")

df = get_songs_kor(i, 10, k)
print("완료했습니당 :)")

KeyboardInterrupt: 

- [1990년대] -> 1000곡
  - i = 4
  - j = 1 ~ 10
  - k = 1

최종 - 2400 + 1000 = 3400

In [ ]:
# 1990년대  ---------- 완료 ----------

i = 4
k = 1

# df = get_songs_kor(i, 1, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 2, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 3, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 4, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 5, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 6, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 7, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 8, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 9, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 10, k)
# print("완료했습니당 :)")

완료했습니당 :)
완료했습니당 :)


- [1980년대] -> 1000곡
  - i = 5
  - j = 1 ~ 10
  - k = 1

1983부터 50곡만 존재

최종 - 3400 + 800 = 4200

In [14]:
# 1980년대  ---------- 완료 ----------
i = 5
k = 1

# df = get_songs_kor(i, 1, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 2, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 3, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 4, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 5, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 6, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 7, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 8, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 9, k)
# print("완료했습니당 :)")

# df = get_songs_kor(i, 10, k)
# print("완료했습니당 :)")

완료했습니당 :)
완료했습니당 :)
완료했습니당 :)
완료했습니당 :)
완료했습니당 :)
완료했습니당 :)
완료했습니당 :)


In [12]:
df

,연도,제목,가수,가사,발매년도,장르,좋아요 수
0,1986,가까이 하기엔 너무 먼 당신,이광조,아 당신은\n당신은 누구시길래\n내 맘 깊은 곳에\n외로움 심으셨나요\n그냥 스처 ...,1989,발라드,"1,293"
1,1986,난 아직 모르잖아요,이문세,세월이 흘러가면\n어디로 가는지\n나는 아직 모르잖아요\n그대 내 곁에 있어요\n떠...,1987,발라드,"1,607"
2,1986,아픈만큼 성숙해지고,구창모,한번쯤 겪어야만 될 사랑의 고통이라면\n그대로 따르겠어요 아무런 이유도 없이\n바라...,1986,록/메탈,362
3,1986,허공,조용필,꿈이었다고 생각하기엔\n너무나도 아쉬움 남아\n가슴태우며 기다리기엔\n너무나도 멀어...,1985,성인가요/트로트,"3,187"
4,1986,지난이야기 (Remake),김완선,가지말아야 했어 그날 그때 그 자리\n자꾸만 떠오르는 그대 예쁜 그 얼굴\n돌담길 ...,2002,댄스,70


### 외국 노래들

- [2020년대] -> 400곡
  - i = 1
  - j = 1 ~ 4
  - k = 3

최종 - 400

In [ ]:
# 2020년대 -------------------------------

i = 1
k = 3

df = get_songs_pop(i, 1, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 2, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 3, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 4, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 5, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 6, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 7, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 8, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 9, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 10, k)
print("완료했습니당 :)")

- [2010년대] -> 1000곡
  - i = 2
  - j = 1 ~ 10
  - k = 3

2012년부터 50곡만 존재

최종 - 400 + 850 = 1250

In [ ]:
# 2010년대 -------------------------------

i = 2
k = 3

df = get_songs_pop(i, 1, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 2, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 3, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 4, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 5, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 6, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 7, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 8, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 9, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 10, k)
print("완료했습니당 :)")

- [2000년대] -> 500
  - i = 3
  - j = 1 ~ 5 // 6 ~ 10
  - k = 3 // 2
  
최종 - 1250 + 500 = 1750

In [ ]:
# 2000년대 (1) -------------------------------

i = 3
k = 3

df = get_songs_pop(i, 1, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 2, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 3, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 4, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 5, k)
print("완료했습니당 :)")

In [ ]:
# 2000년대 (2) -------------------------------

i = 3
k = 2

df = get_songs_pop(i, 6, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 7, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 8, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 9, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 10, k)
print("완료했습니당 :)")

- [1990년대] -> 500곡
  - i = 4
  - j = 1 ~ 10
  - k = 2

최종 - 1750 + 500 = 2250

In [ ]:
# 1990년대  -------------------------------

i = 4
k = 2

df = get_songs_pop(i, 1, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 2, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 3, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 4, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 5, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 6, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 7, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 8, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 9, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 10, k)
print("완료했습니당 :)")

- [1980년대] -> 500
  - i = 5
  - j = 1 ~ 10
  - k = 2

최종 - 2250 + 500 = 2750

In [ ]:
# 1980년대  -------------------------------

i = 5
k = 2

df = get_songs_pop(i, 1, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 2, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 3, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 4, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 5, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 6, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 7, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 8, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 9, k)
print("완료했습니당 :)")

df = get_songs_pop(i, 10, k)
print("완료했습니당 :)")